<a href="https://colab.research.google.com/github/bilal-074/DoctroX/blob/master/Densnet_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flask-ngrok
!pip install opencv-python
!pip install requests
 
 
from google.colab import drive
# drive.flush_and_unmount()
drive.mount('/content/drive')
# Check is Link to Drive is OK
!ls "/content/drive/My Drive/Densenet"
# !if [ -d 'drive/' ]; then echo "Connection to Google drive successful" ; else echo "Error to connect to Google drive"; fi
# !if [ -d 'drive/My Drive/CarCrashDetection/Dataset/train' ]; then echo "Directory train already exist" ; else mkdir 'drive/My Drive/Car' && echo "Directory train created"; fi

Mounted at /content/drive
Data  Figures  FinalModel  Log	Models	testimage  XR_HUMERUS


**Data Loader**

In [ ]:
import os
import numpy as np
import cv2
import random
import keras.backend as K
import requests
def load_path(root_path = '/content/drive/My Drive/Densenet/Data/Train/XR_HUMERUS', size = 512): #given path is a placeholder data only
    '''
    load MURA dataset
 
    '''
 
    Path = []
    labels = []
    for root,dirs,files in os.walk(root_path): #Read all pictures, os.walk Return to iterator genertor Traverse all files
        for name in files:
            path_1 = os.path.join(root,name)
            Path.append(path_1)
            if root.split('_')[-1]=='positive':  #positive Label is 1，otherwise 0；
                labels+=[1]                  #Last level directory file patient11880\\study1_negative\\image3.png
            else:
                labels+=[0]
    print (len(Path))
    labels = np.asarray(labels)
    return Path, labels
 
def load_image(Path = '/content/drive/My Drive/Densenet/Data/Train/XR_HUMERUS', size = 512): #given path is a placeholder data only
    Images = []
    for path in Path:
        try:
            resp = requests.get(path, stream=True).raw
            image = np.asarray(bytearray(resp.read()), dtype="uint8")
            image = cv2.imdecode(image, cv2.IMREAD_GRAYSCALE)   
            # image = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
            image = cv2.resize(image,(size,size))
            image = randome_rotation_flip(image,size)
            Images.append(image)
 
        except Exception as e:
            print(str(e))
 
    Images = np.asarray(Images).astype('float32')
 
    mean = np.mean(Images)          #normalization
    std = np.std(Images)
    Images = (Images - mean) / std
    
    if K.image_data_format() == "channels_first":
        Images = np.expand_dims(Images,axis=1)         #Extended dimension 1
    if K.image_data_format() == "channels_last":
        Images = np.expand_dims(Images,axis=3)             #Extended dimension 3(usebackend tensorflow:aixs=3; theano:axixs=1) 
    return Images
 
 
def randome_rotation_flip(image,size = 512):
    if random.randint(0,1):
        iamge = cv2.flip(image,1) # 1-->horizontal flip 0-->Vertical flip -1-->Horizontal and vertical
 
    if random.randint(0,1):
        angle = random.randint(-30,30)
        M = cv2.getRotationMatrix2D((size/2,size/2),angle,1)
        #The third parameter: the size of the transformed image
        image = cv2.warpAffine(image,M,(size,size))
    return image
 
 
 
if __name__ == '__main__':
    load_path()

1272


**Densenet**

In [ ]:
from keras.models import Model
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import AveragePooling2D
from keras.layers.pooling import GlobalAveragePooling2D
from keras.layers import Input, Concatenate
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
import keras.backend as K
 
 
 
 
def conv_factory(x, concat_axis, nb_filter, dropout_rate=None, weight_decay=1E-4):
 
    x = BatchNormalization(axis=concat_axis, gamma_regularizer=l2(weight_decay), beta_regularizer=l2(weight_decay))(x)
    x = Activation('relu')(x)
    x = Conv2D(nb_filter, (1, 1), kernel_initializer="he_uniform", padding="same", use_bias=False, kernel_regularizer=l2(weight_decay))(x)
    x = BatchNormalization(axis=concat_axis, gamma_regularizer=l2(weight_decay), beta_regularizer=l2(weight_decay))(x)
    x = Activation('relu')(x)
    x = Conv2D(nb_filter, (3, 3), kernel_initializer="he_uniform", padding="same", use_bias=False, kernel_regularizer=l2(weight_decay))(x)
    if dropout_rate:
        x = Dropout(dropout_rate)(x)
 
    return x
 
 
def transition(x, concat_axis, nb_filter, dropout_rate=None, weight_decay=1E-4):
 
    x = BatchNormalization(axis=concat_axis, gamma_regularizer=l2(weight_decay), beta_regularizer=l2(weight_decay))(x)
    x = Activation('relu')(x)
    x = Conv2D(nb_filter, (1, 1), kernel_initializer="he_uniform", padding="same", use_bias=False, kernel_regularizer=l2(weight_decay))(x)
    if dropout_rate:
        x = Dropout(dropout_rate)(x)
    x = AveragePooling2D((2, 2), strides=(2, 2))(x)
 
    return x
 
 
def denseblock(x, concat_axis, nb_layers, nb_filter, growth_rate, dropout_rate=None, weight_decay=1E-4):
 
 
    list_feat = [x]
 
    for i in range(nb_layers):
        x = conv_factory(x, concat_axis, growth_rate,
                         dropout_rate, weight_decay)
        list_feat.append(x)
        x = Concatenate(axis=concat_axis)(list_feat)
        nb_filter += growth_rate
        #print (nb_filter)
 
    return x, nb_filter
 
 
def denseblock_altern(x, concat_axis, nb_layers, nb_filter, growth_rate, dropout_rate=None, weight_decay=1E-4):
 
 
    for i in range(nb_layers):
        merge_tensor = conv_factory(x, concat_axis, growth_rate,  dropout_rate, weight_decay)
        x = Concatenate(axis=concat_axis)([merge_tensor, x])
        nb_filter += growth_rate
 
    return x, nb_filter
 
 
def DenseNet(nb_classes, img_dim, depth, nb_dense_block, growth_rate, nb_filter, dropout_rate=None, weight_decay=1E-4):
 
    if K.image_data_format() == "channels_first":
        concat_axis = 1
    elif K.image_data_format() == "channels_last":
        concat_axis = -1
 
    model_input = Input(shape=img_dim)
 
    assert (depth - 4) % 3 == 0, "Depth must be 3 N + 4"
 
    # layers in each dense block
    nb_layers = int((depth - 4) / 3)
 
    # Initial convolution
    x = Conv2D(nb_filter, (7, 7), strides=(2, 2), kernel_initializer="he_uniform", padding="same", name="initial_conv2D", use_bias=False, kernel_regularizer=l2(weight_decay))(model_input)
 
    # Add dense blocks
    nb_layers1 = [6,12,32,32,48,32,48,64,32]  #3*3 convolutional layer of each denseblock ，
    for block_idx in range(nb_dense_block - 1):
        x, nb_filter = denseblock(x, concat_axis, nb_layers1[block_idx], nb_filter, growth_rate, dropout_rate=dropout_rate, weight_decay=weight_decay)
        # add transition
        x = transition(x, concat_axis,nb_filter, dropout_rate=dropout_rate, weight_decay=weight_decay)
 
    # The last denseblock does not have a transition
    x, nb_filter = denseblock(x, concat_axis, nb_layers1[nb_dense_block-1], nb_filter, growth_rate, dropout_rate=dropout_rate, weight_decay=weight_decay)
 
    x = BatchNormalization(axis=concat_axis, gamma_regularizer=l2(weight_decay), beta_regularizer=l2(weight_decay))(x)
    x = Activation('relu')(x)
    x = GlobalAveragePooling2D(data_format=K.image_data_format())(x)
    x = Dense(nb_classes, activation='sigmoid', kernel_regularizer=l2(weight_decay), bias_regularizer=l2(weight_decay))(x)
 
    densenet = Model(inputs=[model_input], outputs=[x], name="DenseNet")
 
    return densenet

**Main**

In [ ]:
from __future__ import print_function

import os
import time
import datetime
import random
import json
import argparse

import numpy as np
import keras.backend as K

from keras.optimizers import Adam
from keras.utils import np_utils



def run_MURA(batch_size, nb_epoch, depth, nb_dense_block, nb_filter, growth_rate, dropout_rate, learning_rate, weight_decay, plot_architecture):
# data processing
    
    im_size = 320   #Test modification parameters size root_path nb_epoch nb_dense_block         
    X_train_path, Y_train = load_path(root_path = '/content/drive/My Drive/Densenet/Data/Train/XR_HUMERUS',size = im_size)  
    X_valid_path, Y_valid = load_path(root_path = '/content/drive/My Drive/Densenet/Data/Valid/XR_HUMERUS', size = im_size)  

    X_valid = load_image(X_valid_path,im_size)  #Load verification set ahead of time
    Y_valid = np.asarray(Y_valid)
    nb_classes = 1                                
    img_dim = (im_size,im_size,1)                 #Plus the last dimension, type is tuple

# model

    model = DenseNet(nb_classes, img_dim, depth, nb_dense_block, growth_rate, nb_filter, dropout_rate=dropout_rate, weight_decay=weight_decay)
    # Model output
    model.summary()

    # Build optimizer
    opt = Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=["accuracy"])

    if plot_architecture:
        from keras.utils import plot_model
        plot_model(model, to_file='/content/drive/My Drive/Densenet/Figures/densenet_archi.png', show_shapes=True)

# network training

    print("Start Training")

    list_train_loss = []
    list_valid_loss = []
    list_learning_rate = []
    best_record = [100,0,100,100] #Recording optimal [verification set loss function value, accuracy rate, training set data set loss difference,acc difference]
    start_time = datetime.datetime.now()
    for e in range(nb_epoch):

        if e == int(0.25 * nb_epoch):
            K.set_value(model.optimizer.lr, np.float32(learning_rate / 10.))

        if e == int(0.5 * nb_epoch):
            K.set_value(model.optimizer.lr, np.float32(learning_rate / 50.))

        if e == int(0.75 * nb_epoch):
            K.set_value(model.optimizer.lr, np.float32(learning_rate / 100.))


        split_size = batch_size
        num_splits = len(X_train_path) / split_size

        arr_all = np.arange(len(X_train_path)).astype(int)
        random.shuffle(arr_all)                 #Randomly disrupted index order
        arr_splits = np.array_split(arr_all, num_splits)

        l_train_loss = []
        batch_train_loss = []
        start = datetime.datetime.now()

        for i,batch_idx in enumerate(arr_splits):


            X_batch_path,Y_batch = [],[]
            for idx in batch_idx:
                X_batch_path.append(X_train_path[idx])
                Y_batch.append(Y_train[idx])
            X_batch = load_image(Path = X_batch_path, size =im_size)
            Y_batch = np.asarray(Y_batch)
            train_logloss, train_acc = model.train_on_batch(X_batch, Y_batch)

            l_train_loss.append([train_logloss, train_acc])
            batch_train_loss.append([train_logloss, train_acc])
            if i %100 == 0:
                loss_1, acc_1 = np.mean(np.array(l_train_loss), 0)
                loss_2, acc_2 = np.mean(np.array(batch_train_loss), 0)
                batch_train_loss = []           #Current 100 batch loss function and accuracy
                print ('[Epoch {}/{}] [Batch {}/{}] [Time: {}] [all_batchs--> train_epoch_logloss: {:.5f}, train_epoch_acc:{:.5f}] '.format(e+1,nb_epoch,i, len(arr_splits),datetime.datetime.now() - start,loss_1,acc_1), '[this_100_batchs-->train_batchs_logloss: {:.5f}, train_batchs_acc:{:.5f}]'.format(loss_2, acc_2))

        # Run verification set
        valid_logloss, valid_acc = model.evaluate(X_valid, Y_valid, verbose=0, batch_size=64)
        list_train_loss.append(np.mean(np.array(l_train_loss), 0).tolist())
        list_valid_loss.append([valid_logloss, valid_acc])
        list_learning_rate.append(float(K.get_value(model.optimizer.lr)))
        # to convert numpy array to json serializable
        print('[Epoch %s/%s] [Time: %s, Total_time: %s]' % (e + 1, nb_epoch, datetime.datetime.now() - start, datetime.datetime.now() - start_time),end = '')
        print('[train_loss_and_acc:{:.5f} {:.5f}] [valid_loss_acc:{:.5f} {:.5f}]'.format(list_train_loss[-1][0], list_train_loss[-1][1],list_valid_loss[-1][0],list_valid_loss[-1][1]))


        d_log = {}
        d_log["batch_size"] = batch_size
        d_log["nb_epoch"] = nb_epoch
        d_log["optimizer"] = opt.get_config()
        d_log["train_loss"] = list_train_loss
        d_log["valid_loss"] = list_valid_loss
        d_log["learning_rate"] = list_learning_rate

        json_file = os.path.join('/content/drive/My Drive/Densenet/Log/experiment_log_MURA.json')
        with open(json_file, 'w') as fp:
          json.dump(str(d_log), fp, indent=4, sort_keys=True)



        record = [valid_logloss,valid_acc,abs(valid_logloss-list_train_loss[-1][0]),abs(valid_acc-list_train_loss[-1][1]),]
        if ((record[0]<=best_record[0]) &(record[1]>=best_record[1])) :
          if e <= int(0.25 * nb_epoch)|(record[2]<=best_record[2])&(record[3]<=best_record[3]):#Add a difference judgment after a quarter epoch
            best_record=record                      #Record the smallest [validation set loss function value, accuracy rate, training set data loss difference, acc difference]
            print('saving the best model:epoch',e+1,best_record)
            model.save('/content/drive/My Drive/Densenet/Models/best_MURA_modle@epochs{}.h5'.format(e+1))
        
        if (e % 5 == 0):
          model.save('/content/drive/My Drive/Densenet/Models/MURA_modle@epochs{}.h5'.format(e+1))

if __name__ == '__main__':
    batch_size = 8
    nb_epoch = 100
    depth = 6*3+4
    nb_dense_block = 4
    nb_filter = 16
    growth_rate =12
    dropout_rate = 0.2
    learning_rate = 1E-3
    weight_decay = 1E-4
    plot_architecture = False

    list_dir = ["/content/drive/My Drive/Densenet/Log", "/content/drive/My Drive/Densenet/Figures", "/content/drive/My Drive/Densenet/Models"]
    for d in list_dir:
        if not os.path.exists(d):
            os.makedirs(d)

    run_MURA(batch_size, nb_epoch, depth, nb_dense_block,  nb_filter, growth_rate, dropout_rate, learning_rate, weight_decay, plot_architecture)

1272
288
Invalid URL '/content/drive/My Drive/Densenet/Data/Valid/XR_HUMERUS/patient11699/study1_negative/image1.png': No schema supplied. Perhaps you meant http:///content/drive/My Drive/Densenet/Data/Valid/XR_HUMERUS/patient11699/study1_negative/image1.png?
Invalid URL '/content/drive/My Drive/Densenet/Data/Valid/XR_HUMERUS/patient11699/study1_negative/image2.png': No schema supplied. Perhaps you meant http:///content/drive/My Drive/Densenet/Data/Valid/XR_HUMERUS/patient11699/study1_negative/image2.png?
Invalid URL '/content/drive/My Drive/Densenet/Data/Valid/XR_HUMERUS/patient11700/study1_negative/image2.png': No schema supplied. Perhaps you meant http:///content/drive/My Drive/Densenet/Data/Valid/XR_HUMERUS/patient11700/study1_negative/image2.png?
Invalid URL '/content/drive/My Drive/Densenet/Data/Valid/XR_HUMERUS/patient11700/study1_negative/image1.png': No schema supplied. Perhaps you meant http:///content/drive/My Drive/Densenet/Data/Valid/XR_HUMERUS/patient11700/study1_negative

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:234: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:195: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)


AxisError: ignored

In [ ]:
import os
import time
import datetime
import random
import json
import cv2
import numpy as np
import keras.backend as K
 
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.models import load_model
from keras.models import model_from_json
 
from flask_ngrok import run_with_ngrok
from flask import Flask,jsonify,request
import requests
app = Flask(__name__)
run_with_ngrok(app)
im_size = 320
 
 
@app.route("/")
def main():
    url =  request.args.get('imageUrl')
    # Character = request.args.get('character')
   
 
    # url = r'https://www.startradiology.com/uploads/images/english-class-x-shoulder-fig6-walking-stick-configuration-blanco.jpg'
    print(url)
    resp = requests.get(url, stream=True).raw
    image = np.asarray(bytearray(resp.read()), dtype="uint8")
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)
 
 
    model = DenseNet(nb_classes=1, img_dim=(320,320,1), depth=22, nb_dense_block=4, growth_rate=12, nb_filter=16, dropout_rate=0.2, weight_decay=1E-4)
 
    model.load_weights('/content/drive/My Drive/Densenet/FinalModel/MURA_modle@epochs52.h5')
 
    X_valid_path, Y_valid = load_path(root_path = '/content/drive/My Drive/Densenet/testimage/', size = im_size)
    print(f"X VALID PATH:  {X_valid_path}")
    print(f"X VALID PATH TYPE:  {type(X_valid_path)}")
 
    X_valid = load_image([url],im_size)
    y1 = model.predict(X_valid, batch_size=None, verbose=0, steps=None)
    print(f"X VALID :  {X_valid}")
    print(f"X VALID TYPE:  {type(X_valid)}")
    j = len(y1)
 
    for i in range (0, j):
      print(f"y1[i] {y1[i]}")
      print(type(y1[i]))
      if y1[i]>0.5 :
        return (jsonify({"status":"Broken","confidence":round(float(y1[i][0]*100),2)}))
      else:
        return (jsonify({"status":"Intact","confidence":round(float(y1[i][0]*100),2)}))
 
 
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://eab279b4810a.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
https://i.ibb.co/RccsJyY/f02cf242a957.jpg
1
X VALID PATH:  ['/content/drive/My Drive/Densenet/testimage/test.png']
X VALID PATH TYPE:  <class 'list'>


127.0.0.1 - - [01/Jan/2021 05:00:08] "GET /?imageUrl=https://i.ibb.co/RccsJyY/f02cf242a957.jpg HTTP/1.1" 200 -


X VALID :  [[[[ 1.6028889e-01]
   [ 5.7349700e-02]
   [-7.9902552e-02]
   ...
   [-1.7955557e+00]
   [-1.7955557e+00]
   [-1.7955557e+00]]

  [[ 2.7466577e-01]
   [ 1.4885120e-01]
   [ 1.6126134e-04]
   ...
   [-1.7955557e+00]
   [-1.7955557e+00]
   [-1.7955557e+00]]

  [[ 3.5472959e-01]
   [ 2.0603964e-01]
   [ 4.5912012e-02]
   ...
   [-1.7955557e+00]
   [-1.7955557e+00]
   [-1.7955557e+00]]

  ...

  [[ 1.1210546e+00]
   [ 1.1210546e+00]
   [ 1.1210546e+00]
   ...
   [-1.0292306e+00]
   [-1.0521060e+00]
   [-1.0749813e+00]]

  [[ 1.1210546e+00]
   [ 1.1210546e+00]
   [ 1.1210546e+00]
   ...
   [-1.0292306e+00]
   [-1.0521060e+00]
   [-1.0749813e+00]]

  [[ 1.1210546e+00]
   [ 1.1210546e+00]
   [ 1.1210546e+00]
   ...
   [-1.0406684e+00]
   [-1.0635437e+00]
   [-1.0864191e+00]]]]
X VALID TYPE:  <class 'numpy.ndarray'>
y1[i] [0.6669588]
<class 'numpy.ndarray'>
https://i.ibb.co/RccsJyY/f02cf242a957.jpg
1
X VALID PATH:  ['/content/drive/My Drive/Densenet/testimage/test.png']
X VALID PAT

127.0.0.1 - - [01/Jan/2021 05:27:02] "GET /?imageUrl=https://i.ibb.co/RccsJyY/f02cf242a957.jpg HTTP/1.1" 200 -


X VALID :  [[[[-1.705571]
   [-1.705571]
   [-1.705571]
   ...
   [-1.705571]
   [-1.705571]
   [-1.705571]]

  [[-1.705571]
   [-1.705571]
   [-1.705571]
   ...
   [-1.705571]
   [-1.705571]
   [-1.705571]]

  [[-1.705571]
   [-1.705571]
   [-1.705571]
   ...
   [-1.705571]
   [-1.705571]
   [-1.705571]]

  ...

  [[-1.705571]
   [-1.705571]
   [-1.705571]
   ...
   [-1.705571]
   [-1.705571]
   [-1.705571]]

  [[-1.705571]
   [-1.705571]
   [-1.705571]
   ...
   [-1.705571]
   [-1.705571]
   [-1.705571]]

  [[-1.705571]
   [-1.705571]
   [-1.705571]
   ...
   [-1.705571]
   [-1.705571]
   [-1.705571]]]]
X VALID TYPE:  <class 'numpy.ndarray'>
y1[i] [0.5851608]
<class 'numpy.ndarray'>


In [ ]:
pip install opencv-python